In [62]:
import pandas as pd
import numpy as np
hash_size = 2
import math
import operator
X = pd.read_csv("../data/dolphins/dolphins.csv",encoding = 'utf8',sep='\s+',header=None)
y = pd.read_csv("../data/dolphins/dolphins_label.csv",encoding = 'utf8',sep='\s+',names=['label'])
input_dim = X.shape[1]

In [49]:
hash_table = dict()
projections = np.random.randn(hash_size, input_dim)

In [23]:
def generateHash(dataPoint):
    bools = (dataPoint > 0).astype('int')
    #print(''.join(bools.astype('str')))
    return ''.join(bools.astype('str'))

In [4]:
def hashItem(dataPoint, label):
        hash_value = generateHash(dataPoint)
        hash_table[hash_value] = hash_table\
            .get(hash_value, list()) + [label]

In [5]:
def getHashItem(dataPoint):
        hash_value = generateHash(dataPoint)
        return hash_table.get(hash_value, [])

In [6]:
def cosine_sim(hash_value1,hash_value2):
    return np.dot(hash_value1,hash_value2)(np.linalg.norm(hash_value1))(np.linalg.norm(hash_value2))

In [7]:
temp=[]

for row in X.iterrows():
    index, data = row
    temp.append(data.tolist())

In [8]:
for i in range(X.shape[0]):
    dataPoint = temp[i]
    #print(generateHash(dataPoint))
    hashItem(dataPoint,y['label'][i])

In [13]:
#hash_table
dataPoint = temp[6]
#print(generateHash(dataPoint))
print(generateHash(dataPoint))


1100


In [15]:
print(len(temp))


62


In [19]:
dictionary={}
for index, item in enumerate(temp):
    hashvalue = generateHash(item)
    dictionary[hashvalue]=dictionary\
            .get(hashvalue, list()) + [index]

In [20]:
dictionary

{'1100': [0,
  1,
  3,
  5,
  6,
  16,
  17,
  18,
  19,
  21,
  32,
  33,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  44,
  45,
  46,
  53,
  54,
  56,
  59,
  60],
 '1000': [2, 4],
 '1101': [7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  20,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  34,
  35,
  43,
  49,
  50,
  51,
  55,
  57,
  58,
  61],
 '1110': [22, 23, 47, 48, 52],
 '1001': [31]}

In [1]:
class LSH:
    def __init__(self, num_tables, hash_size, inp_dimensions):
        self.num_tables = num_tables
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_tables = list()
        for i in range(self.num_tables):
            self.hash_tables.append(HashTable(self.hash_size, self.inp_dimensions))
    
    def __setitem__(self, inp_vec, label):
        for table in self.hash_tables:
            table[inp_vec] = label
    
    def __getitem__(self, inp_vec):
        results = list()
        for table in self.hash_tables:
            results.extend(table[inp_vec])
        return list(set(results))

In [7]:
def Distance(x, y):
    distance = 0
    for i in range(len(x)-1):
        distance += pow((x[i]-y[i]), 2)
    return math.sqrt(distance)

In [57]:
def kNeighbors(data, query, k):
    distances = []
    neighbors = []
   
    for i in range(len(data)):
        distance = Distance(data[i], query)
        distances.append((data[i], distance))
    distances.sort(key=operator.itemgetter(1))
    #print(type(distances))
    #print(distances)
    #print(distances)
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors


In [9]:
def Predict(neighbors):
    predicted = {}
    for i in range(len(neighbors)):
        output = neighbors[i][-1]
        if output in predicted:
            predicted[output] += 1
        else:
            predicted[output] = 1
    pre = sorted(predicted.items(), key=operator.itemgetter(1), reverse=True)
    #print(pre)
    return pre[0][0]

In [ ]:
def main(i,train_data,test_data,y_test):
    y_pred=[]
    k = i
    for i in range(len(test_data)):
        neighbors = kNeighbors(train_data, test_data[i], k)
        result = Predict(neighbors)
        y_pred.append(result)
    accuracy = accuracy_score(y_test, y_pred)
    f1scoremacro = f1_score(y_test, y_pred,average='macro')
    f1scoremicro = f1_score(y_test, y_pred,average='micro')
    return accuracy,f1scoremacro,f1scoremicro

In [63]:
from sklearn.cross_validation import KFold

k=1
projections = np.random.randn(input_dim,hash_size, )
transformation_matrix=np.asmatrix(projections)
data=np.asmatrix(X)
X = data * transformation_matrix

X = np.array(X)
y = np.array(y)
kf = KFold(y.shape[0],n_folds=10)

for train_index, validation_index in kf:
            
    X_train, X_val = X[train_index], X[validation_index]
    y_train, y_val = y[train_index], y[validation_index]
    df_train = pd.concat([pd.DataFrame(X), pd.DataFrame(y_train)],axis = 1)
    df_test = pd.concat([pd.DataFrame(X_val),pd.DataFrame( y_val)],axis = 1)
    train_data = df_train.values.tolist()
    val_data = df_test.values.tolist()
    train_data=np.asarray(train_data)
    #accuracy,f1_scoremacro,f1_scoremicro = main(3,train_data,val_data,y_val)
    dictionary={}
    
    
    for index, item in enumerate(X_train):
        #send same projection matrix to generatehash function
        hashvalue = generateHash(item)
        dictionary[hashvalue]=dictionary\
                .get(hashvalue, list()) + [index]
    data=[]  
    y_pred=[]
    #print(type(train_data))
    #print(type(X_train))
    for index,item in enumerate(X_val):
        hashvalue = generateHash(item)
        for i in dictionary[hashvalue]:
            data.append(train_data[i])
        neighbors = kNeighbors(data, val_data[index], k)
        result = Predict(neighbors)
        y_pred.append(result)
    print("------------")
    print(y_val)
    print("predicted")
    print(y_pred)      
        #for each i in dictionary[hashvalue] #hashvalue obtained from above
        # calculate euclidean distance between x_train[i] and item for 5 nn return the top 5 with least distance
        #and finally take the majority vote
        #note that X_train[i] and item are the projected data


------------
[[0]
 [0]
 [0]
 [0]
 [0]
 [2]
 [0]]
predicted
[1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0]
------------
[[1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]]
predicted
[1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0]
------------
[[1]
 [0]
 [0]
 [0]
 [0]
 [2]]
predicted
[2.0, 2.0, 2.0, 2.0, 1.0, 1.0]
------------
[[2]
 [2]
 [2]
 [2]
 [1]
 [1]]
predicted
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
------------
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]]
predicted
[0.0, 2.0, 0.0, 1.0, 2.0, 2.0]
------------
[[0]
 [2]
 [0]
 [1]
 [2]
 [2]]
predicted
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
------------
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]]
predicted
[0.0, 0.0, 2.0, 2.0, 2.0, 1.0]
------------
[[0]
 [0]
 [2]
 [2]
 [2]
 [1]]
predicted
[1.0, 1.0, 2.0, 2.0, 2.0, 1.0]
------------
[[1]
 [1]
 [2]
 [2]
 [2]
 [1]]
predicted
[0.0, 3.0, 0.0, 0.0, 0.0, 1.0]
------------
[[0]
 [3]
 [0]
 [0]
 [0]
 [1]]
predicted
[0.0, 1.0, 0.0, 0.0, 0.0, 1.0]


In [ ]:
def generateHash(dataPoint):
    bools = (np.dot(dataPoint,projections.T) > 0).astype('int')
    #print(''.join(bools.astype('str')))
    return ''.join(bools.astype('str'))

In [ ]:
def calculateHamming()

In [23]:
from sklearn.cross_validation import KFold

k=1


X = np.array(X)
y = np.array(y)
kf = KFold(y.shape[0],n_folds=10)

for train_index, validation_index in kf:
            
    X_train, X_val = X[train_index], X[validation_index]
    y_train, y_val = y[train_index], y[validation_index]
    
    dictionary={}
    
    
    for index, item in enumerate(X_train):
        #send same projection matrix to generatehash function
        hashvalue = generateHash(item)
        dictionary[hashvalue]=y_train[index]
   
    y_pred=[]
    #print(type(train_data))
    #print(type(X_train))
    for index,item in enumerate(X_val):
        hashvalue = generateHash(item)
        for i in dictionary[hashvalue]:
            data.append(train_data[i])
        neighbors = kNeighbors(data, val_data[index], k)
        result = Predict(neighbors)
        y_pred.append(result)
    print("------------")
    print(y_val)
    print("predicted")
    print(y_pred)

NameError: name 'hello' is not defined